In [1]:
#%pip install ipyleaflet requests openai

In [17]:
import requests
from IPython.display import display
from ipyleaflet import Map as LeafletMap, DrawControl
from transformers import pipeline

In [3]:
# Once data has been processed, we check if there are any invalid data (i.e. marker is on land)
def is_valid_data(data):
    hourly_data = data['hourly']

    for hourly_entry in hourly_data.items():
        values = hourly_entry[1]
        if all(value is None for value in values):
            return False

    return True

In [12]:
# Initialize llm with transformers library
llm = pipeline("summarization", model="facebook/bart-large-cnn")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [28]:
# Clean the data before passing to llm
def clean_data(data):
    clean_data = ""
    clean_data += "Location latitude: " + str(data['latitude']) + ", longitude: " + str(data['longitude']) + "\n"
    clean_data += "Timezone: " + data['timezone'] + " " + data['timezone_abbreviation'] + "\n"
    clean_data += "Hourly Wave Heights: " + ", ".join([str(height) for height in data['hourly']['wave_height']]) + " meters.\n"
    clean_data += "Hourly Wave Direction: " + ", ".join([str(direction) for direction in data['hourly']['wave_direction']]) + " kts.\n"
    clean_data += "Hourly Wave Period: " + ", ".join([str(period) for period in data['hourly']['wave_period']]) + " s.\n"
    clean_data += "Hourly Ocean Current Velocity: " + ", ".join([str(period) for period in data['hourly']['ocean_current_velocity']]) + " km/h.\n"
    clean_data += "Hourly Ocean Current Direction: " + ", ".join([str(period) for period in data['hourly']['ocean_current_direction']]) + " kts.\n"
    return clean_data

In [31]:
# Event handler when user pins marker on map
def handle_marker(event, action, geo_json):
    coordinates = geo_json['geometry']['coordinates']
    longitude, latitude = str(coordinates[0]), str(coordinates[1])

    # Fetch sea conditions from Open-Meteo API
    url = "https://marine-api.open-meteo.com/v1/marine?latitude=" + latitude + "&longitude=" + longitude + "&hourly=wave_height,wave_direction,wave_period,ocean_current_velocity,ocean_current_direction&daily=wave_height_max,wave_direction_dominant,wave_period_max&timezone=auto"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if is_valid_data(data):
            clean_summary = clean_data(data)
            summary = llm(clean_summary, max_length=130, min_length=30, do_sample=False)
            print(summary)
        else:
            print("You can't cruise on land!")
    else:
        print("Error fetching data from Open-Meteo API: " + str(response.status_code))

In [32]:
map_center = [20.0, 0.0]
m = LeafletMap(center=map_center, zoom=2, scroll_wheel_zoom=True)

draw_control = DrawControl(
    marker={'shapeOptions': {}},
    polyline={},
    polygon={},
    circle={},
    circlemarker={},
)
m.add_control(draw_control)
draw_control.on_draw(handle_marker)

# Display map
display(m)

Map(center=[20.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

IndexError: index out of range in self